In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
os.chdir('/content/drive/My Drive/ECG_Heartbeat_Classification')

In [ ]:
!pip install -r /content/drive/MyDrive/ECG_Heartbeat_Classification/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.1/30.1 MB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 57.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of scipy to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
import pandas as pd
import numpy as np
import random
import os
from plotly.offline import plot
import plotly.graph_objs as go
from collections import Counter

# Đảm bảo lưu file vào thư mục `ECG_Heartbeat_Classification/images/` đúng vị trí
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Lấy thư mục cha của `scripts`
image_dir = os.path.join(base_dir, "images")  # Đặt thư mục lưu file vào `images/`
os.makedirs(image_dir, exist_ok=True)  # Tạo thư mục nếu chưa có

df = pd.read_csv("./input/mitbih_test.csv", header=None)
print(df.shape)
print(Counter(df[187].values))

Y = np.array(df[187].values).astype(np.int8)
X = np.array(df[list(range(187))].values)

indexes = random.sample(list(range(df.shape[0])), 10)

for i in indexes:
    data = [go.Scatter(
              x=list(range(187)),
              y=X[i, :])]

    plot({"data": data,
          "layout": {"title": "Heartbeat Class : %s " % Y[i]}}, filename=os.path.join(image_dir, f"{i}.html"))


(21892, 188)
Counter({0.0: 18118, 4.0: 1608, 2.0: 1448, 1.0: 556, 3.0: 162})


In [12]:
import pandas as pd
import numpy as np

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import f1_score, accuracy_score


df_train = pd.read_csv("./input/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("./input/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


def get_model():
    nclass = 5
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()

file_path = "./models/train_mitbih_cnn.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 187, 1)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_8 (Conv1D)                    │ (None, 183, 16)             │              96 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_9 (Conv1D)                    │ (None, 179, 16)             │           1,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 89, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 89, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_10 (Conv1D)                   │ (None, 87, 32)              │           1,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, 85, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 42, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 42, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_12 (Conv1D)                   │ (None, 40, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_13 (Conv1D)                   │ (None, 38, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_5 (MaxPooling1D)       │ (None, 19, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 19, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_14 (Conv1D)                   │ (None, 17, 256)             │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_15 (Conv1D)                   │ (None, 15, 256)             │         196,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_1               │ (None, 256)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3_mitbih (Dense)               │ (None, 5)                   │             325 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 254,901 (995.71 KB)

 Trainable params: 254,901 (995.71 KB)

 Non-trainable params: 0 (0.00 B)

...

2463/2463 - 11s - 5ms/step - acc: 0.9806 - loss: 0.0659 - val_acc: 0.9837 - val_loss: 0.0596 - learning_rate: 0.0010
Epoch 18/1000

Epoch 18: val_acc did not improve from 0.98367
2463/2463 - 20s - 8ms/step - acc: 0.9809 - loss: 0.0653 - val_acc: 0.9829 - val_loss: 0.0611 - learning_rate: 0.0010
Epoch 19/1000

Epoch 19: val_acc did not improve from 0.98367
2463/2463 - 9s - 4ms/step - acc: 0.9816 - loss: 0.0626 - val_acc: 0.9822 - val_loss: 0.0630 - learning_rate: 0.0010
Epoch 20/1000

Epoch 20: val_acc did not improve from 0.98367

Epoch 20: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
2463/2463 - 10s - 4ms/step - acc: 0.9817 - loss: 0.0624 - val_acc: 0.9813 - val_loss: 0.0626 - learning_rate: 0.0010
Epoch 21/1000

Epoch 21: val_acc improved from 0.98367 to 0.98550, saving model to ./models/train_mitbih_cnn.h5


2463/2463 - 11s - 4ms/step - acc: 0.9862 - loss: 0.0446 - val_acc: 0.9855 - val_loss: 0.0526 - learning_rate: 1.0000e-04
Epoch 22/1000

Epoch 22: val_acc improved from 0.98550 to 0.98664, saving model to ./models/train_mitbih_cnn.h5


2463/2463 - 11s - 4ms/step - acc: 0.9879 - loss: 0.0399 - val_acc: 0.9866 - val_loss: 0.0509 - learning_rate: 1.0000e-04
Epoch 23/1000

Epoch 23: val_acc did not improve from 0.98664
2463/2463 - 19s - 8ms/step - acc: 0.9877 - loss: 0.0398 - val_acc: 0.9866 - val_loss: 0.0512 - learning_rate: 1.0000e-04
Epoch 24/1000

Epoch 24: val_acc improved from 0.98664 to 0.98675, saving model to ./models/train_mitbih_cnn.h5


2463/2463 - 11s - 4ms/step - acc: 0.9886 - loss: 0.0377 - val_acc: 0.9868 - val_loss: 0.0512 - learning_rate: 1.0000e-04
Epoch 25/1000

Epoch 25: val_acc did not improve from 0.98675
2463/2463 - 20s - 8ms/step - acc: 0.9890 - loss: 0.0355 - val_acc: 0.9868 - val_loss: 0.0509 - learning_rate: 1.0000e-04
Epoch 26/1000

Epoch 26: val_acc did not improve from 0.98675
2463/2463 - 9s - 4ms/step - acc: 0.9889 - loss: 0.0354 - val_acc: 0.9868 - val_loss: 0.0513 - learning_rate: 1.0000e-04
Epoch 27/1000

Epoch 27: val_acc did not improve from 0.98675

Epoch 27: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
2463/2463 - 11s - 4ms/step - acc: 0.9892 - loss: 0.0348 - val_acc: 0.9864 - val_loss: 0.0498 - learning_rate: 1.0000e-04
Epoch 28/1000

Epoch 28: val_acc improved from 0.98675 to 0.98744, saving model to ./models/train_mitbih_cnn.h5


2463/2463 - 11s - 5ms/step - acc: 0.9899 - loss: 0.0321 - val_acc: 0.9874 - val_loss: 0.0504 - learning_rate: 1.0000e-05
Epoch 29/1000

Epoch 29: val_acc did not improve from 0.98744
2463/2463 - 10s - 4ms/step - acc: 0.9902 - loss: 0.0326 - val_acc: 0.9865 - val_loss: 0.0503 - learning_rate: 1.0000e-05
Epoch 30/1000

Epoch 30: val_acc did not improve from 0.98744
2463/2463 - 10s - 4ms/step - acc: 0.9899 - loss: 0.0316 - val_acc: 0.9870 - val_loss: 0.0505 - learning_rate: 1.0000e-05
Epoch 31/1000

Epoch 31: val_acc did not improve from 0.98744

Epoch 31: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
2463/2463 - 9s - 4ms/step - acc: 0.9897 - loss: 0.0327 - val_acc: 0.9868 - val_loss: 0.0500 - learning_rate: 1.0000e-05
Epoch 32/1000

Epoch 32: val_acc did not improve from 0.98744
2463/2463 - 11s - 4ms/step - acc: 0.9897 - loss: 0.0324 - val_acc: 0.9868 - val_loss: 0.0501 - learning_rate: 1.0000e-06
Epoch 33/1000

Epoch 33: val_acc did not improve from 0.98744
2463/24

In [13]:
import pandas as pd
import numpy as np

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

df_1 = pd.read_csv("./input/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("./input/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


def get_model():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "./models/train_ptbdb_cnn.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 187, 1)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_16 (Conv1D)                   │ (None, 183, 16)             │              96 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_17 (Conv1D)                   │ (None, 179, 16)             │           1,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_6 (MaxPooling1D)       │ (None, 89, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 89, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_18 (Conv1D)                   │ (None, 87, 32)              │           1,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_19 (Conv1D)                   │ (None, 85, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_7 (MaxPooling1D)       │ (None, 42, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 42, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_20 (Conv1D)                   │ (None, 40, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_21 (Conv1D)                   │ (None, 38, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_8 (MaxPooling1D)       │ (None, 19, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 19, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_22 (Conv1D)                   │ (None, 17, 256)             │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_23 (Conv1D)                   │ (None, 15, 256)             │         196,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_2               │ (None, 256)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3_ptbdb (Dense)                │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 254,641 (994.69 KB)

 Trainable params: 254,641 (994.69 KB)

 Non-trainable params: 0 (0.00 B)

...

328/328 - 3s - 8ms/step - acc: 0.9611 - loss: 0.1049 - val_acc: 0.9734 - val_loss: 0.0750 - learning_rate: 0.0010
Epoch 12/1000

Epoch 12: val_acc did not improve from 0.97339
328/328 - 2s - 6ms/step - acc: 0.9627 - loss: 0.0985 - val_acc: 0.9554 - val_loss: 0.1084 - learning_rate: 0.0010
Epoch 13/1000

Epoch 13: val_acc did not improve from 0.97339
328/328 - 1s - 4ms/step - acc: 0.9672 - loss: 0.0921 - val_acc: 0.9691 - val_loss: 0.0682 - learning_rate: 0.0010
Epoch 14/1000

Epoch 14: val_acc did not improve from 0.97339

Epoch 14: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
328/328 - 1s - 4ms/step - acc: 0.9672 - loss: 0.0918 - val_acc: 0.9562 - val_loss: 0.1124 - learning_rate: 0.0010
Epoch 15/1000

Epoch 15: val_acc improved from 0.97339 to 0.97768, saving model to ./models/train_ptbdb_cnn.h5


328/328 - 1s - 4ms/step - acc: 0.9801 - loss: 0.0572 - val_acc: 0.9777 - val_loss: 0.0549 - learning_rate: 1.0000e-04
Epoch 16/1000

Epoch 16: val_acc improved from 0.97768 to 0.98026, saving model to ./models/train_ptbdb_cnn.h5


328/328 - 3s - 8ms/step - acc: 0.9830 - loss: 0.0493 - val_acc: 0.9803 - val_loss: 0.0539 - learning_rate: 1.0000e-04
Epoch 17/1000

Epoch 17: val_acc did not improve from 0.98026
328/328 - 2s - 5ms/step - acc: 0.9849 - loss: 0.0429 - val_acc: 0.9777 - val_loss: 0.0479 - learning_rate: 1.0000e-04
Epoch 18/1000

Epoch 18: val_acc improved from 0.98026 to 0.98112, saving model to ./models/train_ptbdb_cnn.h5


328/328 - 2s - 6ms/step - acc: 0.9840 - loss: 0.0446 - val_acc: 0.9811 - val_loss: 0.0482 - learning_rate: 1.0000e-04
Epoch 19/1000

Epoch 19: val_acc improved from 0.98112 to 0.98798, saving model to ./models/train_ptbdb_cnn.h5


328/328 - 2s - 6ms/step - acc: 0.9859 - loss: 0.0421 - val_acc: 0.9880 - val_loss: 0.0437 - learning_rate: 1.0000e-04
Epoch 20/1000

Epoch 20: val_acc did not improve from 0.98798
328/328 - 2s - 7ms/step - acc: 0.9861 - loss: 0.0412 - val_acc: 0.9845 - val_loss: 0.0435 - learning_rate: 1.0000e-04
Epoch 21/1000

Epoch 21: val_acc did not improve from 0.98798
328/328 - 1s - 4ms/step - acc: 0.9863 - loss: 0.0397 - val_acc: 0.9811 - val_loss: 0.0460 - learning_rate: 1.0000e-04
Epoch 22/1000

Epoch 22: val_acc did not improve from 0.98798

Epoch 22: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
328/328 - 1s - 4ms/step - acc: 0.9859 - loss: 0.0422 - val_acc: 0.9828 - val_loss: 0.0441 - learning_rate: 1.0000e-04
Epoch 23/1000

Epoch 23: val_acc did not improve from 0.98798
328/328 - 1s - 4ms/step - acc: 0.9872 - loss: 0.0336 - val_acc: 0.9863 - val_loss: 0.0410 - learning_rate: 1.0000e-05
Epoch 24/1000

Epoch 24: val_acc did not improve from 0.98798
328/328 - 1s - 4ms/st

In [15]:
import pandas as pd
import numpy as np

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

df_1 = pd.read_csv("./input/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("./input/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


def get_model():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid", trainable=False)(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid", trainable=False)(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()
file_path = "./models/transfer_learning_ptbdb_freeze.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early
model.load_weights("./models/train_mitbih_cnn.h5", by_name=True)
model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 187, 1)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_32 (Conv1D)                   │ (None, 183, 16)             │              96 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_33 (Conv1D)                   │ (None, 179, 16)             │           1,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_12 (MaxPooling1D)      │ (None, 89, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 89, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_34 (Conv1D)                   │ (None, 87, 32)              │           1,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_35 (Conv1D)                   │ (None, 85, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_13 (MaxPooling1D)      │ (None, 42, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 42, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_36 (Conv1D)                   │ (None, 40, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_37 (Conv1D)                   │ (None, 38, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_14 (MaxPooling1D)      │ (None, 19, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 19, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_38 (Conv1D)                   │ (None, 17, 256)             │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_39 (Conv1D)                   │ (None, 15, 256)             │         196,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_4               │ (None, 256)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3_ptbdb (Dense)                │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 254,641 (994.69 KB)

 Trainable params: 20,673 (80.75 KB)

 Non-trainable params: 233,968 (913.94 KB)

Epoch 1/1000

Epoch 1: val_acc improved from -inf to 0.69957, saving model to ./models/transfer_learning_ptbdb_freeze.h5


328/328 - 13s - 40ms/step - acc: 0.7244 - loss: 0.5894 - val_acc: 0.6996 - val_loss: 0.6117 - learning_rate: 0.0010
Epoch 2/1000

Epoch 2: val_acc did not improve from 0.69957
328/328 - 1s - 3ms/step - acc: 0.7244 - loss: 0.5824 - val_acc: 0.6996 - val_loss: 0.6073 - learning_rate: 0.0010
Epoch 3/1000

Epoch 3: val_acc did not improve from 0.69957
328/328 - 1s - 4ms/step - acc: 0.7244 - loss: 0.5720 - val_acc: 0.6996 - val_loss: 0.5944 - learning_rate: 0.0010
Epoch 4/1000

Epoch 4: val_acc did not improve from 0.69957

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
328/328 - 1s - 3ms/step - acc: 0.7254 - loss: 0.5561 - val_acc: 0.6996 - val_loss: 0.5722 - learning_rate: 0.0010
Epoch 5/1000

Epoch 5: val_acc did not improve from 0.69957
328/328 - 1s - 4ms/step - acc: 0.7266 - loss: 0.5441 - val_acc: 0.6996 - val_loss: 0.5682 - learning_rate: 1.0000e-04
Epoch 6/1000

Epoch 6: val_acc did not improve from 0.69957
328/328 - 1s - 4ms/step - acc: 0.7277 - loss: 

In [16]:
import pandas as pd
import numpy as np

from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

df_1 = pd.read_csv("./input/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("./input/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


def get_model():
    nclass = 1
    inp = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)

    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    dense_1 = Dense(64, activation=activations.relu, name="dense_2")(dense_1)
    dense_1 = Dense(nclass, activation=activations.sigmoid, name="dense_3_ptbdb")(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    model.summary()
    return model

model = get_model()

file_path = "./models/transfer_learning_ptbdb_full.h5"

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early
model.load_weights("./models/train_mitbih_cnn.h5", by_name=True)
model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)

pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)           │ (None, 187, 1)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_40 (Conv1D)                   │ (None, 183, 16)             │              96 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_41 (Conv1D)                   │ (None, 179, 16)             │           1,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_15 (MaxPooling1D)      │ (None, 89, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 89, 16)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_42 (Conv1D)                   │ (None, 87, 32)              │           1,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_43 (Conv1D)                   │ (None, 85, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_16 (MaxPooling1D)      │ (None, 42, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 42, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_44 (Conv1D)                   │ (None, 40, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_45 (Conv1D)                   │ (None, 38, 32)              │           3,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_17 (MaxPooling1D)      │ (None, 19, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_22 (Dropout)                 │ (None, 19, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_46 (Conv1D)                   │ (None, 17, 256)             │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_47 (Conv1D)                   │ (None, 15, 256)             │         196,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_5               │ (None, 256)                 │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_23 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3_ptbdb (Dense)                │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 254,641 (994.69 KB)

 Trainable params: 254,641 (994.69 KB)

 Non-trainable params: 0 (0.00 B)

...

328/328 - 1s - 4ms/step - acc: 0.9550 - loss: 0.1139 - val_acc: 0.9751 - val_loss: 0.0735 - learning_rate: 0.0010
Epoch 12/1000

Epoch 12: val_acc did not improve from 0.97511
328/328 - 3s - 8ms/step - acc: 0.9620 - loss: 0.1008 - val_acc: 0.9648 - val_loss: 0.0823 - learning_rate: 0.0010
Epoch 13/1000

Epoch 13: val_acc did not improve from 0.97511
328/328 - 2s - 7ms/step - acc: 0.9625 - loss: 0.0989 - val_acc: 0.9708 - val_loss: 0.0766 - learning_rate: 0.0010
Epoch 14/1000

Epoch 14: val_acc did not improve from 0.97511

Epoch 14: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
328/328 - 1s - 4ms/step - acc: 0.9693 - loss: 0.0825 - val_acc: 0.9700 - val_loss: 0.0687 - learning_rate: 0.0010
Epoch 15/1000

Epoch 15: val_acc improved from 0.97511 to 0.97940, saving model to ./models/transfer_learning_ptbdb_full.h5


328/328 - 4s - 11ms/step - acc: 0.9775 - loss: 0.0576 - val_acc: 0.9794 - val_loss: 0.0488 - learning_rate: 1.0000e-04
Epoch 16/1000

Epoch 16: val_acc improved from 0.97940 to 0.98026, saving model to ./models/transfer_learning_ptbdb_full.h5


328/328 - 1s - 4ms/step - acc: 0.9813 - loss: 0.0514 - val_acc: 0.9803 - val_loss: 0.0482 - learning_rate: 1.0000e-04
Epoch 17/1000

Epoch 17: val_acc improved from 0.98026 to 0.98197, saving model to ./models/transfer_learning_ptbdb_full.h5


328/328 - 1s - 4ms/step - acc: 0.9834 - loss: 0.0482 - val_acc: 0.9820 - val_loss: 0.0445 - learning_rate: 1.0000e-04
Epoch 18/1000

Epoch 18: val_acc improved from 0.98197 to 0.98627, saving model to ./models/transfer_learning_ptbdb_full.h5


328/328 - 1s - 4ms/step - acc: 0.9846 - loss: 0.0413 - val_acc: 0.9863 - val_loss: 0.0406 - learning_rate: 1.0000e-04
Epoch 19/1000

Epoch 19: val_acc did not improve from 0.98627
328/328 - 2s - 7ms/step - acc: 0.9839 - loss: 0.0438 - val_acc: 0.9828 - val_loss: 0.0431 - learning_rate: 1.0000e-04
Epoch 20/1000

Epoch 20: val_acc did not improve from 0.98627
328/328 - 1s - 4ms/step - acc: 0.9859 - loss: 0.0412 - val_acc: 0.9828 - val_loss: 0.0423 - learning_rate: 1.0000e-04
Epoch 21/1000

Epoch 21: val_acc did not improve from 0.98627

Epoch 21: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
328/328 - 1s - 4ms/step - acc: 0.9855 - loss: 0.0402 - val_acc: 0.9837 - val_loss: 0.0403 - learning_rate: 1.0000e-04
Epoch 22/1000

Epoch 22: val_acc did not improve from 0.98627
328/328 - 2s - 5ms/step - acc: 0.9840 - loss: 0.0374 - val_acc: 0.9837 - val_loss: 0.0384 - learning_rate: 1.0000e-05
Epoch 23/1000

Epoch 23: val_acc did not improve from 0.98627
328/328 - 2s - 5ms/st